<a href="https://colab.research.google.com/github/OtavioCarvalho1/Time_Series/blob/main/time_series_SHM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

Importação do sinal utilizado no TREINAMENTO

In [5]:
Names = ['tempo [s]', 'Amplitude PZT Atuador', 'Amplitude PZT Sensor']

df_train = pd.read_csv("/content/drive/MyDrive/Dataset-LambWave-Wagner/PL0402_Ref/PL0402_Ref2_1_150KHz__5ciclos_B3_A3.csv", names=Names)
df_train

,tempo [s],Amplitude PZT Atuador,Amplitude PZT Sensor
0,0.000000e+00,-0.000312,-0.000000e+00
1,1.000000e-07,-0.000136,-0.000000e+00
2,2.000000e-07,-0.000141,-0.000000e+00
3,3.000000e-07,-0.000105,-0.000000e+00
4,4.000000e-07,-0.000038,-1.000000e-08
...,...,...,...
14995,1.499500e-03,-0.000429,-3.192000e-05
14996,1.499600e-03,-0.000536,-2.153000e-05
14997,1.499700e-03,-0.000430,-9.680000e-06
14998,1.499800e-03,-0.000518,3.530000e-06


Importação do sinal utilizado no TESTE

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Dataset-LambWave-Wagner/PL0402_Ref/PL0402_Ref2_2_150KHz__5ciclos_B3_A3.csv", names=Names)
df_test